In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#imports
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt




%matplotlib inline

In [ ]:
df = pd.read_csv('../input/consumer_complaints/Consumer_Complaints.csv')

In [ ]:
df.info()

#### We will cluster documents on Consumer complaint and product columns

In [ ]:
df['Consumer Complaint'].dropna(axis=0, inplace=True)

In [ ]:
def process_data(complaint):

    cleaned_complaint = re.sub("[^a-zA-Z]", " ", complaint)
    
    text = sent_tokenize(cleaned_complaint)
    
    cleaned_complaint = re.sub("[^a-zA-Z]", " ", complaint)
    
    words = cleaned_complaint.lower().split()
    
    stops = set(stopwords.words('english'))
    
    words = [w for w in words if not w in stops]
    
    return " ".join(words)

In [ ]:
data = [process_data(i) for i in df['Consumer Complaint']]

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, 
                             norm='l2', encoding='latin-1', 
                             ngram_range=(1, 2),
                             max_features=500,
                             stop_words=None,
                            tokenizer=None)

In [ ]:
features = vectorizer.fit_transform(data).toarray()

In [ ]:
pca = PCA(n_components=2, random_state=42)
trans_df = pca.fit_transform(features)

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(trans_df[:,0], trans_df[:, 1], s=60)
plt.show

In [ ]:
n_clusters = range(1, 20, 2)
scores = []

for i in n_clusters:
    print(i)
    kmeans = KMeans(n_clusters=i, init='k-means++', n_jobs=-1)
    kmeans.fit(trans_df)
    scores.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(n_clusters, scores, '-o')
plt.xlabel("Number of clusters")
plt.ylabel("Score")
plt.title("Elbow Curve")

In [ ]:
Kmeans = KMeans(n_clusters=10)
Kmeans.fit(trans_df)
predicted = Kmeans.predict(trans_df)

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(trans_df[:, 0], trans_df[:,1], c=predicted)
centers = Kmeans.cluster_centers_
plt.scatter(centers[:,0], centers[:,1],  marker='x', s=200)
plt.show()